In [1]:
#| default_exp  process

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/app')
sys.path.insert(0,'..')

In [2]:
#| export
from audio.redis import *
from audio.audio import *

from pathlib import Path
import json

import uuid

import pandas as pd

import redis.asyncio as aioredis
from dataclasses import dataclass
import numpy as np
import asyncio

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
# import uuid
# client_id = str(uuid.uuid4());client_id

In [5]:

redis_inner_client = asyncio.run(get_inner_redis())
client_id = '851f343e-4954-4f0a-8835-9664cc91c181'

In [8]:
#| export
async def diarize(audio_data, client_id, audio_name, shift):
    redis_inner_client = await get_inner_redis()
    audio = Audio(chunk_name=audio_name, redis_client=redis_inner_client, data=audio_data)
    await audio.save()
    await redis_inner_client.lpush('Audio2DiarizeQueue', f'{audio_name}:{client_id}')
    done = await redis_inner_client.brpop(f'DiarizeReady:{audio_name}')
    diarization = Diarisation(audio_name, redis_inner_client)
    await diarization.get()
    df = pd.DataFrame(diarization.data)
    if len(df)>0:
        df['silence'] = df['start']-df['end'].shift()
        df['speaker_change'] = df['speaker'] != df['speaker'].shift()
        df['len'] = df['end'] - df['start']
        df = df[df['len'] > 0.5]
        df['speaker_change'] = np.where(df['silence']>2,True,df['speaker_change'])

        df['speaker_change'] = df['speaker_change'].cumsum()
        df = df.groupby('speaker_change').agg({'speaker': 'first', 'start': 'first', 'end': 'last'})
        df['conv_start'] = df['start'] + shift
        df['conv_end'] = df['end'] + shift
        return df.to_dict('records')
    else: return df

In [9]:
#| export

import time
async def transcribe(diarization_result,connection_id,audio_name):
    redis_inner_client = await get_inner_redis()
    for segment in diarization_result:
        segment['connection_id'] = connection_id
        segment['name'] = str(uuid.uuid4())
        segment['audio_id'] = audio_name
        print('lpush')
        await redis_inner_client.lpush('TranscribeQueue', json.dumps(segment))
        print('brpop')
        done = await redis_inner_client.brpop(f'TranscribeReady:{segment["name"]}')
        print('done')
        transcription = Transcript(segment["name"],redis_inner_client)
        await transcription.get()
        segment['transcription'] = transcription.data
        await redis_inner_client.lpush(f'Segment:{connection_id}', json.dumps(segment))

In [10]:
#| export
async def get_next_chunk_start(diarization_result, length):
    if len(diarization_result)>0:
        last_speech = diarization_result[-1]
        if round(last_speech['end']) == length:
            return last_speech['conv_start']
        

        else: return last_speech['conv_end']



    else: return None

In [11]:
#from file

#path = Path('/app/audio').ls()[1]
path = Path('/app/testdata/david_audio.webm')
audio_slicer =await AudioSlicer.from_file(path,'webm')
connection_id = path.name

In [12]:
#audio_slicer.slice(117, 180)

In [13]:
#| export
async def get_next_chunk_start(diarization_result, length):
    if len(diarization_result)>0:
        last_speech = diarization_result[-1]
        if last_speech['end']+0.1 > length:
            return last_speech['conv_start']
        
        else: 
            return last_speech['conv_end']



    else: return None

In [18]:
start = 0
length = 60
end = start+length

In [19]:
connection_id = 'david_audio.webm'#'david_audio.webm'

In [20]:
try:
    while True:
        print(start,end)
        audio_data = await audio_slicer.export_data(start,end)
        audio_name = str(uuid.uuid4())
        audio = Audio(chunk_name=audio_name, redis_client=redis_inner_client, data=audio_data)
        await audio.save()
        print('waiting diarize')
        diarization_result = await diarize(audio_data,client_id,audio_name,start)
        print('waiting transcribe')
        await transcribe(diarization_result,connection_id,audio_name)
        start_ = await get_next_chunk_start(diarization_result,length)
        start = start_ if start_ else start+length
        if end>audio_slicer.audio.duration_seconds:break
        end+=length
except KeyboardInterrupt:
    pass

0 60
waiting diarize
done
waiting transcribe
done
done
done
done
done
done
53.573853989813244 120
waiting diarize
done
waiting transcribe
done
done
done
done
done
done
117.45331069609509 180
waiting diarize
done
waiting transcribe
done
done
done
done
done
done
179.0067911714771 240
waiting diarize
done
waiting transcribe
done
done
done
done
212.69949066213923 300
waiting diarize
done
waiting transcribe
done
done
done
done
done
done
done
291.2648556876061 360
waiting diarize
done
waiting transcribe
done
done
done
done
done
352.86926994906617 420
waiting diarize
done
waiting transcribe
done
done
done
done
412.7419354838709 480
waiting diarize
done
waiting transcribe
done
done
done
436.29881154499145 540
waiting diarize
done
waiting transcribe
done
done
done
done
done
528.2597623089982 600
waiting diarize
done
waiting transcribe
done
done
573.7860780984719 660
waiting diarize
done
waiting transcribe
done
done
612.334465195246 720
waiting diarize
done
waiting transcribe
done
done
done
693.